#Dataset

In [2]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

path = "FULL"

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,

                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 3976 images belonging to 7 classes.


#VGG16

In [5]:
VGG16=tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None
)
VGG16.trainable = True
print(len(VGG16.layers))

19


In [6]:
i = 0
for layer in VGG16.layers:
  if i < 11:
    layer.trainable = False
  i+=1

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

In [8]:
input = tf.keras.layers.Input(shape=(224, 224, 3))
x = data_augmentation(input)
x = VGG16(input, training=False)
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
VGG16 = tf.keras.models.Model(inputs = input, outputs = x, name = "VGG16")

VGG16.summary()

Model: "VGG16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 average_pooling2d (AverageP  (None, 4, 4, 512)        0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 7)                 57351     
                                                                 
Total params: 14,772,039
Trainable params: 13,036,551
Non-trainable params: 1,735,488
_________________________________________

In [9]:
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

VGG16.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = VGG16.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=100,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/100
31/31 [==============================] - 473s 15s/step - loss: 5.1437 - accuracy: 0.2040 - val_loss: 1.8717 - val_accuracy: 0.2121
Epoch 2/100
31/31 [==============================] - 466s 15s/step - loss: 1.8587 - accuracy: 0.2495 - val_loss: 1.8583 - val_accuracy: 0.2165
Epoch 3/100
31/31 [==============================] - 466s 15s/step - loss: 1.8657 - accuracy: 0.2319 - val_loss: 1.8408 - val_accuracy: 0.2433
Epoch 4/100
31/31 [==============================] - 470s 15s/step - loss: 1.8473 - accuracy: 0.2278 - val_loss: 1.8360 - val_accuracy: 0.2478
Epoch 5/100
31/31 [==============================] - 467s 15s/step - loss: 1.8701 - accuracy: 0.2157 - val_loss: 1.8566 - val_accuracy: 0.2321
Epoch 6/100
31/31 [==============================] - 469s 15s/step - loss: 1.8540 - accuracy: 0.2440 - val_loss: 1.8833 - val_accuracy: 0.1987
Epoch 7/100
31/31 [==============================] - 467s 15s/step - loss: 1.8509 - accuracy: 0.2329 - val_loss: 1.7936 - val_accuracy: 0.2879

In [12]:
VGG16.save('VGG16Augmented_Model.keras')

In [5]:
new_model = tf.keras.models.load_model('VGG16Augmented_Model.keras')

# loss, acc = new_model.evaluate(validation_set)
# print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [6]:
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

new_model.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = new_model.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=60,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/60
31/31 [==============================] - 423s 13s/step - loss: 1.1457 - accuracy: 0.5708 - val_loss: 1.2889 - val_accuracy: 0.5000
Epoch 2/60
31/31 [==============================] - 406s 13s/step - loss: 1.1118 - accuracy: 0.5675 - val_loss: 1.2348 - val_accuracy: 0.5357
Epoch 3/60
31/31 [==============================] - 405s 13s/step - loss: 1.0628 - accuracy: 0.5993 - val_loss: 1.2520 - val_accuracy: 0.5469
Epoch 4/60
31/31 [==============================] - 403s 13s/step - loss: 1.0701 - accuracy: 0.5958 - val_loss: 1.2356 - val_accuracy: 0.5223
Epoch 5/60
31/31 [==============================] - 402s 13s/step - loss: 1.0954 - accuracy: 0.5832 - val_loss: 1.2318 - val_accuracy: 0.5201
Epoch 6/60
31/31 [==============================] - 399s 13s/step - loss: 1.1015 - accuracy: 0.5870 - val_loss: 1.2689 - val_accuracy: 0.4955
Epoch 7/60
31/31 [==============================] - 402s 13s/step - loss: 1.0791 - accuracy: 0.5887 - val_loss: 1.2750 - val_accuracy: 0.5201
Epoch 

In [7]:
new_model.save('new_model.keras')

In [5]:
new_model = tf.keras.models.load_model('new_model.keras')
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

new_model.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = new_model.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=30,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/30
31/31 [==============================] - 456s 15s/step - loss: 0.8780 - accuracy: 0.6773 - val_loss: 1.3999 - val_accuracy: 0.5402
Epoch 2/30
31/31 [==============================] - 473s 15s/step - loss: 0.8262 - accuracy: 0.6976 - val_loss: 1.3892 - val_accuracy: 0.5156
Epoch 3/30
31/31 [==============================] - 444s 14s/step - loss: 0.8492 - accuracy: 0.6930 - val_loss: 1.2449 - val_accuracy: 0.5491
Epoch 4/30
31/31 [==============================] - 448s 14s/step - loss: 0.7835 - accuracy: 0.7046 - val_loss: 1.2843 - val_accuracy: 0.5379
Epoch 5/30
31/31 [==============================] - 470s 15s/step - loss: 0.8278 - accuracy: 0.6910 - val_loss: 1.3695 - val_accuracy: 0.4643
Epoch 6/30
31/31 [==============================] - 455s 15s/step - loss: 0.8058 - accuracy: 0.7011 - val_loss: 1.3336 - val_accuracy: 0.5647
Epoch 7/30
31/31 [==============================] - 462s 15s/step - loss: 0.8155 - accuracy: 0.7026 - val_loss: 1.3175 - val_accuracy: 0.5603
Epoch 

In [7]:
new_model.save('new_model.keras')

In [6]:
new_model = tf.keras.models.load_model('new_model.keras')
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

new_model.compile(optimizer = 'adam',
      
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = new_model.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=60,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )


Epoch 1/60
31/31 [==============================] - 484s 16s/step - loss: 0.7528 - accuracy: 0.7248 - val_loss: 1.2767 - val_accuracy: 0.5848
Epoch 2/60
31/31 [==============================] - 474s 15s/step - loss: 0.6975 - accuracy: 0.7510 - val_loss: 1.5966 - val_accuracy: 0.5156
Epoch 3/60
31/31 [==============================] - 474s 15s/step - loss: 0.7086 - accuracy: 0.7510 - val_loss: 1.4991 - val_accuracy: 0.5201
Epoch 4/60
31/31 [==============================] - 473s 15s/step - loss: 0.7032 - accuracy: 0.7530 - val_loss: 1.4171 - val_accuracy: 0.5558
Epoch 5/60
31/31 [==============================] - 478s 15s/step - loss: 0.6544 - accuracy: 0.7712 - val_loss: 1.5443 - val_accuracy: 0.5379
Epoch 6/60
31/31 [==============================] - 483s 16s/step - loss: 0.6933 - accuracy: 0.7581 - val_loss: 1.4865 - val_accuracy: 0.5603
Epoch 7/60
31/31 [==============================] - 475s 15s/step - loss: 0.6635 - accuracy: 0.7631 - val_loss: 1.6301 - val_accuracy: 0.5223
Epoch 

In [7]:
new_model.save('new_model.keras')

In [6]:
new_model = tf.keras.models.load_model('new_model.keras')
steps_per_epoch = training_set.n // 512
validation_steps = validation_set.n // 512

new_model.compile(optimizer = 'adam',
      
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
batch_size = 40
history = new_model.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=100,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )
new_model.save('new_model.keras')

Epoch 1/100
31/31 [==============================] - 770s 25s/step - loss: 0.5601 - accuracy: 0.8014 - val_loss: 1.7802 - val_accuracy: 0.5491
Epoch 2/100
31/31 [==============================] - 745s 24s/step - loss: 0.5308 - accuracy: 0.8080 - val_loss: 1.7113 - val_accuracy: 0.5268
Epoch 3/100
31/31 [==============================] - 385s 12s/step - loss: 0.4241 - accuracy: 0.8579 - val_loss: 1.8337 - val_accuracy: 0.5290
Epoch 4/100
31/31 [==============================] - 383s 12s/step - loss: 0.5082 - accuracy: 0.8175 - val_loss: 1.7261 - val_accuracy: 0.5335
Epoch 5/100
31/31 [==============================] - 379s 12s/step - loss: 0.4308 - accuracy: 0.8529 - val_loss: 2.2859 - val_accuracy: 0.5201
Epoch 6/100
31/31 [==============================] - 383s 12s/step - loss: 0.4537 - accuracy: 0.8326 - val_loss: 1.8830 - val_accuracy: 0.5335
Epoch 7/100
31/31 [==============================] - 382s 12s/step - loss: 0.4902 - accuracy: 0.8276 - val_loss: 1.7351 - val_accuracy: 0.5402